<a href="https://colab.research.google.com/github/andyarnell/Global_ecological_zones_mapping/blob/main/gez_comparison_sankey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aim: create Sankey plots to illustrate magnitude of climate zone changes to help compare GEZ update options

Clone GitHub repository for visualising results

In [1]:
# Change the current working directory to "/content".
%cd "/content"

# Clone the GitHub repository "sepal_mgci" into the current directory.
# NB 'fatal' error on reruns are typically just saying it already exists
!git clone https://github.com/andyarnell/sankee.git

/content
Cloning into 'sankee'...
remote: Enumerating objects: 1115, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 1115 (delta 248), reused 314 (delta 230), pack-reused 761 (from 1)
Receiving objects: 100% (1115/1115), 15.31 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (653/653), done.


Set up packages

In [2]:
# to automatically reload modules.
%load_ext autoreload

# Set to reload all modules before executing code.
%autoreload 2


# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

# install_if_not_exists("sankee")
install_if_not_exists("os")

# from google.colab import output
# output.enable_custom_widget_manager()
# install_if_not_exists("Sidecar")
# from sidecar import Sidecar

import os

%cd ..

print(os.listdir())

import sys

%cd "/content/sankee"

import sankee

import ee # google earth engine

import geemap

from google.colab import output


gee_project_name = "ee-andyarnellgee"

ee.Authenticate()

ee.Initialize(project=gee_project_name)


os is already installed.
/
['boot', 'bin', 'root', 'sys', 'srv', 'opt', 'var', 'usr', 'proc', 'home', 'mnt', 'lib32', 'media', 'etc', 'libx32', 'lib', 'lib64', 'tmp', 'sbin', 'dev', 'run', 'content', '.dockerenv', 'tools', 'datalab', 'python-apt', 'NGC-DL-CONTAINER-LICENSE', 'cuda-keyring_1.0-1_all.deb']
/content/sankee


Parameters

In [5]:

# choose mask
# where: 0 = no_mask, 1 = gez_mountains_inverse_mask, 2 = mountain_belts_inverse_mask, 3 = combined_mountain_mask
mask_choice = 0 #NB set to 0 unless want to exclude mountainous areas from map/stats

# proportion of points to use for stats (closer to 1 the more precise but slower)
prop_of_points = 0.01


# choose region to assess in the sankey plot at the end
region_index = 1  # numeric value - change as needed based dictionary below

# dictionary mapping region_index to region name
region_dict = {
    0: "Africa",
    1: "E Asia",
    2: "Europe",
    3: "N Asia",
    4: "S Asia",
    5: "Oceania",
    6: "SE Asia",
    7: "SW Asia",
    8: "Australia",
    9: "Caribbean",
    10: "Antarctica",
    11: "S Atlantic",
    12: "Central Asia",
    13: "Indian Ocean",
    14: "North America",
    15: "South America",
    16: "Central America"
}

factor = 10 # to coarsen resolution of the images to be sampled. Where 1 is native resolution. Coarser factors e.g. 10, should be quicker as 10*10 fewer pixels, but less accurate.


# Define the band name
band = "b1" # keep as b1 unless reason to change

# Define class labels and colors corresponding to each pixel value.
labels = {
    1: "tropical",
    2: "subtropical",
    3: "temperate",
    4: "boreal",
    5: "polar",
}

# colours
palette = {
    1: "#BA1A00",
    2: "#F09136",
    3: "#FFDF4A",
    4: "#009F49",
    5: "#1452A1",
}
#NB colours aligned with https://www.nature.com/articles/s41893-024-01416-5/figures/2 and using https://redketchup.io/color-picker

Input data prep

In [7]:
# input data

chelsa_climate_1981_2010 = ee.ImageCollection("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/chelsa_tas_1981_2010");
chelsa_pet = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/chelsa_pet_penman_mean_1981-2010_V_2_1");
worldclim_climate = ee.ImageCollection("WORLDCLIM/V1/MONTHLY");
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE");
mountain_belts = ee.Image("users/xavidelamo/SDG1542_Mntn_BioclimaticBelts");#from Korner et al
hzl_contemp = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/life_zones_contemporary");

point_asset ="users/frarssuser1/WorldESTIMATES6Dec21_V1_GEEv3_/WorldESTIMATES6Dec21_V1_GEEv5_" # RSS points

admin_bounds = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");

# Load the existing GEZ data showing level 2
gez_2010_level2 = ee.Image('users/bornToBeAlive/gez_2010_wgs84').selfMask() # includes removing lakes etc (val 90)
gez_2010_level2 = gez_2010_level2.updateMask(gez_2010_level2.neq(90))

# Load the existing GEZ data showing level 1 (approximate climate zones)
gez_2010_level1 = gez_2010_level2.divide(10).floor().int().selfMask()

# Load the Holdridge Life Zones (adapted from Elsen et al 2022)
hzl_contemp = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/life_zones_contemporary")

# Create a simplified version of the Holdridge Life Zones to approximate GEZ level 1
hzl_contemp_simple = hzl_contemp.divide(10).floor().int().selfMask().updateMask(gez_2010_level1)

hzl_contemp_simple_remap = hzl_contemp_simple.remap([1,2,3,4,5,6],[6,5,4,3,2,1],0) #remap values to match gez etc



#select region based on region_index
region_name = region_dict[region_index]

print(f"Region name: {region_name} (for region_index {region_index})")

aoi = admin_bounds.filter(ee.Filter.eq("wld_rgn",region_name))

print ("number of countries: ",aoi.size().getInfo())



# get resolution from one of the inputs
scale = gez_2010_level1.select(0).projection().nominalScale().multiply(ee.Number(factor)).getInfo()

print ("scale: ", scale)

# functions related to koppen trewartha zone creation

# Function to calculate number of months above a temperature threshold
def calc_number_of_months_above(monthly_image_col, band_name_tavg, temp_threshold):
    def map_threshold(image):
        image_tavg = image.select(band_name_tavg)
        image_gte_thresh = image_tavg.gte(temp_threshold)
        return image_gte_thresh

    monthly_image_col_tavg = monthly_image_col.select(band_name_tavg)
    monthly_image_col_tavg_gte_thresh = monthly_image_col_tavg.map(map_threshold)
    count_of_months_above = monthly_image_col_tavg_gte_thresh.sum()
    return count_of_months_above

# Function to classify Köppen-Trewartha climate zones
def classify_koppen_trewartha(monthly_image_col, band_name_tavg, prec_avg, pet_avg, high_thresh, low_thresh):
    # Calculate number of months above 18 degrees and 10 degrees
    number_of_months_above_18_degrees = calc_number_of_months_above(monthly_image_col, band_name_tavg, high_thresh)
    number_of_months_above_10_degrees = calc_number_of_months_above(monthly_image_col, band_name_tavg, low_thresh)

    # Define Köppen-Trewartha rules
    threshold_18_deg = 12  # standard kt val :12 (but for toggling to get similar to GEZ level 1, 8 is closer!)
    high_threshold_10_deg = 8  # standard kt val :8
    middle_threshold_10_deg = 4  # standard kt val :4
    low_threshold_10_deg = 1  # standard kt val : 1

    # Define Köppen-Trewartha rules
    tropical = number_of_months_above_18_degrees.gte(threshold_18_deg)
    subtropical = number_of_months_above_10_degrees.gte(high_threshold_10_deg).And(number_of_months_above_18_degrees.lt(threshold_18_deg))
    temperate = number_of_months_above_10_degrees.gte(middle_threshold_10_deg).And(number_of_months_above_10_degrees.lt(high_threshold_10_deg))
    boreal = number_of_months_above_10_degrees.gte(low_threshold_10_deg).And(number_of_months_above_10_degrees.lt(middle_threshold_10_deg))
    polar = number_of_months_above_10_degrees.lt(low_threshold_10_deg)

    # Combine rules to create zones
    koppenTrewarthaZones = ee.Image(0) \
        .where(tropical, 1) \
        .where(subtropical, 2) \
        .where(temperate, 3) \
        .where(boreal, 4) \
        .where(polar, 5) \
        .selfMask()

    # Return the image with Köppen-Trewartha zones
    return koppenTrewarthaZones.rename('koppen_trewartha')



# Data prep

# Potential evapotranspiration
terraclimate_pet_avg = terraclimate.select('pet').filter(ee.Filter.date('1960-01-01', '1991-01-01')).mean()

# Precipitation
worldclim_climate_prec_avg = worldclim_climate.select('prec').mean().updateMask(gez_2010_level1)



# Analysis
# Apply classify_koppen_trewartha function to worldclim
worldclim_kt = classify_koppen_trewartha(worldclim_climate,
                                                      'tavg',
                                                      worldclim_climate_prec_avg,
                                                      terraclimate_pet_avg,  # as not included in worldclim
                                                      180,
                                                      100)

# Apply classify_koppen_trewartha function to chelsa
chelsa_kt = classify_koppen_trewartha(chelsa_climate_1981_2010,
                                                      'b1',
                                                      worldclim_climate_prec_avg,  # using for temp interim
                                                      chelsa_pet,
                                                      18,
                                                      10)

# Mask to terrestrial areas (based on worldclim extent for now)
chelsa_kt = chelsa_kt.updateMask(worldclim_kt.gt(0)).updateMask(gez_2010_level1)

gez_mountains = (
    gez_2010_level2.eq(16)
    .Or(gez_2010_level2.eq(25))
    .Or(gez_2010_level2.eq(35))
    .Or(gez_2010_level2.eq(43))
)

gez_2010_level1 = gez_2010_level1.updateMask(chelsa_kt) # all layers should have same extent now

Region name: E Asia (for region_index 1)
number of countries:  13
scale:  1113.1949079327358


Create images and show on map
- switch layers on and off as required

In [8]:
# Create a map
Map = geemap.Map()

# get boundaries for the world

# aoi = ee.Geometry.Polygon([xMin, yMin, xMax, yMax], None,False)
world = ee.Geometry.Polygon([
    [-180, 90], [0, 90], [180, 90],
    [180,-90], [0, -90], [-180,-90]],
    None, False);

# aoi = world
# Map.addLayer(aoi,{'color': 'FF0000'},"aoi")





### apply choice for masking data (no mask vs masking out mountain areas)

gez_mountains_inverse_mask = gez_mountains.eq(0)

# Map.addLayer(gez_mountains_inverse_mask, {}, "gez_mountains_inverse_mask ")

mountain_belts_inverse_mask = mountain_belts.unmask(0).eq(0)

# Map.addLayer(mountain_belts_inverse_mask, {}, "mountain_belts_inverse_mask ")

combined_mountain_mask = ee.ImageCollection(
    # get mask of mountains in both layers
    [mountain_belts_inverse_mask,
    gez_mountains_inverse_mask]).min()

no_mask = ee.Image(1)


# see parameter at start. if it is true then mountains are masked out
mask = [no_mask, gez_mountains_inverse_mask,mountain_belts_inverse_mask,combined_mountain_mask][mask_choice]

Map.addLayer(mask, {}, "mask")

# apply mask and make sure band names are consistent
gez_2010_level1 = gez_2010_level1.select(0).rename(band).updateMask(mask)

worldclim_kt = worldclim_kt.select(0).rename(band).updateMask(mask)

chelsa_kt =chelsa_kt.select(0).rename(band).updateMask(mask)

# hzl_contemp_simple_remap = hzl_contemp_simple_remap.rename("b1").updateMask(mask)

print(min(*palette.keys()))

viz_params_kt_zones = {'min':min(*palette.keys()),'max':max(*palette.keys()),'palette':[*palette.values()]}

###### Add layers to the map
Map.addLayer(gez_2010_level1, viz_params_kt_zones, "gez_2010_level1") #.randomVisualizer(), {}, "gez_2010_level1")

# Add data-driven zones based on climate datasets
Map.addLayer(worldclim_kt,viz_params_kt_zones,'Worldclim Köppen-Trewartha Climate Zones');

Map.addLayer(chelsa_kt,viz_params_kt_zones,'Chelsa Köppen-Trewartha Climate Zones');

# Map.addLayer(hzl_contemp, {}, "life zones contemporary - all")
# Map.addLayer(hzl_contemp_simple_remap.randomVisualizer(), {}, "holdridge life zones contemporary - simple")



# Display the map
Map
# sc = Sidecar(title='Sidecar Output')
# with sc:
#     display(Map)




1


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Make Sankey plot for a given region comparing GEZ to data driven KT outputs

In [ ]:
title="Transitions: GEZ Level 1 to WorldClim KT, Chelsa KT  (from left to right)" # <title is not showing on my browser unfortunately>

image_list = [gez_2010_level1, worldclim_kt, chelsa_kt] # order of images to compare (from left to right)

print ("Region: ", region_dict.get(region_index))
print (title)

# Generate the Sankey diagram from the two images
sankee.sankify(title=title,
    image_list=image_list,
    region=aoi.geometry().bounds(),
    band=band,
    labels=labels,
    palette=palette,
    scale=scale,
    # n=500,
    seed=1,
    point_asset=point_asset, ## uncomment this line if want to use point asset (i.e. RSS plots)
    proportion = prop_of_points # defined in parameters above
)


#Note: To see transition values hover over plot. To download png hover over top right of image until camera icon appears.

Region:  E Asia
Transitions: GEZ Level 1 to WorldClim KT, Chelsa KT  (from left to right)


SankeyPlot()